In [ ]:
! pip install dash

In [ ]:
# Import libraries
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
import dash
import dash_table
from dash_table.Format import Format, Group
import dash_core_components as dcc
import dash_html_components as html
import re
from dash.dependencies import Input, Output
from skimage import io
import glob
#from plotly.subplots import make_subplots
#import base64

pd.options.plotting.backend = "plotly"
#pd.set_option("precision", 2)
#data1['shift'] = round(data1['shift'],2)

# Open data
#with open('secondary_shape_data.pkl', 'rb') as f:
#    data1 = pickle.load(f)
#data1 = pd.DataFrame.from_dict(data1)

with open('all_finite_nuclearity_dataset_string.pkl', 'rb') as f:
    data1 = pickle.load(f)
data1 = pd.DataFrame.from_dict(data1)

with open('convex_hull_data.pkl', 'rb') as file:
    convex_hull_data = pickle.load(file)
convex_hull_data = pd.DataFrame.from_dict(convex_hull_data)

# Find all unique nuclearities
nuc_pairs = data1['nuclearity'].unique()
nuc_pairs_dropdown = []
for i in nuc_pairs:
    nuc_pairs_dropdown.append({'label': i, 'value': i})

# Initialize dash layout elements
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

# Create markdown text for interface
markdown_text1 = '''
 ###### Select active component:
'''
markdown_text2 = '''
###### Select inactive component:
'''

markdown_text3 = '''
###### Select nuclearity:
'''
actives = {'Pd', 'Pt', 'Rh', 'Ru', 'Ag', 'Ir'}
hosts = {'Zn', 'Al', 'Ga', 'In', 'Cd'}
unique_shapes = sorted(data1['graph_id'].unique())

#hull_df = pd.DataFrame.from_dict(convex_hull_data)
#fig = make_subplots(rows=1, cols=1, shared_xaxes=False, shared_yaxes=False)
#fig.add_scatter(y=hull_df.formation_energy, x=hull_df.active_mole_fraction,mode='markers',marker = dict(size = 5),text = hull_df.active_mole_fraction + ', ' + hull_df.formation_energy,hoverinfo='text',row=1,col=1)
#df = px.data.iris() # iris is a pandas DataFrame
#fig = px.scatter(df, x="sepal_width", y="sepal_length")
#fig = px.scatter(convex_hull_data, x="active_mole_fraction", y="formation_energy")

# Create the plotly dash layout
app.layout = html.Div(children=[
    html.Div([
        html.Div([dcc.Link(id='imagelink', href = 'https://www.cmu.edu/brand/brand-guidelines/images/wordmarksquare-red-600x600.png', children = html.Img(src=app.get_asset_url('Fig0.png')))],style={'width': '45%', 'display': 'inline-block'}),#html.H1(children='click here')),
        html.Div([html.H1(children='Nuclearity Zoo')], style={'width': '48%','display': 'inline-block'})],
    ),
    dcc.Tabs([
        dcc.Tab(label='Filters', children=[
            html.Div([
            # Markdown for the element number checklist
                html.Div([
                    html.Div([dcc.Markdown(children=markdown_text1)],style={'width': '40%','display': 'inline-block'}),

                    html.Div([
                        dcc.RadioItems(
                        id='num-active',
                        options=[{'label': i, 'value': i} for i in actives],
                        value='Pd',
                        labelStyle={'display': 'inline-block'}
                        )], style={'width': '40%','display': 'inline-block'})
                ]),

                # Markdown for the element number checklist
                html.Div([
                    html.Div([dcc.Markdown(children=markdown_text2)], style={'width': '40%','display': 'inline-block'}),

                    html.Div([
                        dcc.RadioItems(
                        id='num-inactive',
                        options=[{'label': i, 'value': i} for i in hosts],
                        value='Zn',
                        labelStyle={'display': 'inline-block'}
                        )], style={'width': '40%','display': 'inline-block'})
                ]),

                html.Div([
                    html.Div([dcc.Markdown(children='###### Formation energy range:')], style={'width': '40%','display': 'inline-block'}),

                    html.Div([
                        dcc.RangeSlider(
                            id='form_energy',
                            min=-0.5,
                            max=0.5,
                            step=0.1,
                            marks={
                                -0.5: '-0.5',
                                -0.25: '-0.25',
                                0: '0',
                                0.25: '0.25',
                                0.5: '0.5'
                            },
                            value=[-0.5, 0.5]
                        )], style={'width': '40%','display': 'inline-block'})
                ]),

                html.Div([
                    html.Div([dcc.Markdown(children='###### Active mole fraction:')], style={'width': '40%','display': 'inline-block'}),

                    html.Div([
                        dcc.RangeSlider(
                            id='mol_fr',
                            min=0,
                            max=1,
                            step=0.1,
                            marks={
                                0: '0',
                                0.25: '0.25',
                                0.5: '0.5',
                                0.75: '0.75',
                                1: '1'
                            },
                            value=[0, 1]
                        )], style={'width': '40%','display': 'inline-block'})
                ]),

                html.Div([
                    html.Div([dcc.Markdown(children=markdown_text3)], style={'width': '40%','display': 'inline-block'}),

                    # Dropdown bar so the user may select which scaling pair to view
                    html.Div([
                        dcc.Dropdown(
                            options=[
                                *nuc_pairs_dropdown
                            ],
                            placeholder="Select nuclearity",
                            #value=f'{nuc_pairs[0]}',
                            id = 'nuclearity',
                        style={"width": "100%"}
                        )], style={'width': '40%','display': 'inline-block'}),
                ]),

                dcc.Markdown(children='###### Select shape:'),

                dcc.Checklist(
                    id = 'shape',
                    options=[{'label': i, 'value': i} for i in unique_shapes],
                    value=unique_shapes,
                    labelStyle={'display': 'inline-block'}
                )],
                style = {'width':'50%', 'display':'inline-block'}),

            html.Div(id='hull_plot', style = {'width':'50%', 'display':'inline-block'}),

            html.Div(id = 'image_clubbed', children = html.Img(src=app.get_asset_url('all_shapes_2.png'), style={'height':'30%', 'width':'30%'}))

        ]),

        dcc.Tab(label='Search Results', children=[
            dcc.Markdown('###### '),

            html.Div([
                html.Div([dcc.Markdown('###### Sort by:')], style={'width':'10%','display':'inline-block'}),

                #Dropdown container for deciding what value to sort by
                html.Div([
                    dcc.Dropdown(
                        id='sorting_keys',
                        style={"width": "98%"},
                        options=[
                            {'label': 'Formation Energy', 'value': 'formation_energy'},
                            {'label': 'Graph id', 'value': 'graph_id'},
                            {'label': 'Mole Fraction Active Component', 'value': 'active_mol_fr'},
                            {'label': 'Shift', 'value': 'shift'}
                        ],
                        value='shift'
                    )], style={'width':'20%','display':'inline-block'}),

                #Slider for determining if it should be sorted in ascending or descending order
                html.Div([
                    dcc.RadioItems(
                        id = 'ascdesc',
                        options=[
                            {'label': 'Descending', 'value': 'Descending'},
                            {'label': 'Ascending', 'value': 'Ascending'}
                        ],
                        value='Ascending',
                        labelStyle={'display': 'inline-block'}
                    )], style={'width':'20%','display':'inline-block'})
            ]),

            #html.Div(id='sorting-key-container'),

            #html.Div(id='ascdesc-output-container'),

            html.Div(id = 'results found'),
            html.Div(id='table output')

        ]),
        dcc.Tab(label='About', children=[
            dcc.Markdown('###### Kindly refresh the page if it does not load correctly'),
            dcc.Markdown('######'),
            dcc.Markdown('###### This work is part of a collaborative project between the Ulissi group (Department of Chemical Engineering, Carnegie Mellon University), the Janik group (Department of Chemical Engineering, Pennsylvania State University), the Phases Research Lab (Department of Material Science and Engineering, Pennsylvania State University) and the Rioux group (Department of Chemical Engineering, Pennsylvania State University), funded by the U.S. Department of Energy.'),
            dcc.Markdown('###### Contact us: zulissi@andrew.cmu.edu'),
            html.Div(id = 'DOE', children = html.Img(src=app.get_asset_url('DOE.png'), style={'height':'30%', 'width':'30%'}))
        ])


    ])

])

# Other functions

@app.callback(
    Output('shape','options'),
    Input('nuclearity','value'))

def update_element(nuclearity):
    groups = data1.groupby('nuclearity')
    temp = groups.get_group(nuclearity)
    temp = pd.DataFrame(temp)
    shape_list = sorted(temp['graph_id'].unique())
    options = [{'label': i, 'value': i} for i in shape_list]
    return options


@app.callback(
    Output('hull_plot','children'),
    Input('num-active', 'value'),
    Input('num-inactive', 'value'))

def update_element(act,inact):
    data4 = convex_hull_data[convex_hull_data['active']==act]
    data4 = data4[data4['inactive']==inact]
    data4 = data4[data4['formation_energy']<=-0.1]
    fig = px.scatter(data4, x="active_mole_fraction", y="formation_energy")
    children = dcc.Graph(figure=fig)
    return children


# Create dash callback
# Create dash callback
@app.callback(
    Output('table output', 'children'),
    Output('results found', 'children'),
    Input('num-active', 'value'),
    Input('num-inactive', 'value'),
    Input('nuclearity', 'value'),
    Input('shape', 'value'),
    Input('form_energy', 'value'),
    Input('mol_fr', 'value'),
    Input('ascdesc', 'value'),
    Input('sorting_keys', 'value')
)

# Primary callback function
def update_element(act, inact, nuclearity,shape,energy,mol_fr,upordown,sortkey):
    data2 = pd.DataFrame()
    pd.set_option("display.precision", 2)
    groups = data1.groupby(['active_el', 'inactive_el', 'nuclearity','graph_id']) #Can move around the order of the keys
    #for act in active:
    #    for inact in inactive:
    for sh in shape:
        try:
            temp = groups.get_group((act, inact, nuclearity,sh))
            temp = pd.DataFrame(temp)
            data2 = data2.append(temp)
        except:
            continue
    #data2 = data2.append(data1.iloc[0:5])
    data3 = data2[['bulk','miller_index','shift','nuclearity','graph_id','formation_energy','active_mol_fr','aurl']]
    data3 = data3[data3['formation_energy']>=energy[0]]
    data3 = data3[data3['formation_energy']<=energy[1]]
    data3 = data3[data3['active_mol_fr']>=mol_fr[0]]
    data3 = data3[data3['active_mol_fr']<=mol_fr[1]]
    columns = [{"name": i, "id": i} for i in data3.columns]
    if upordown == 'Ascending':
        order = True
    else:
        order = False
    data3 = data3.sort_values(by=[sortkey], axis=0, ascending=order, inplace=False, kind='quicksort', na_position='last', ignore_index=False, key=None)

    #with open('data3.pkl','wb') as f:
      #pickle.dump(data3, f)
    #columns[3]["format"] = Format(precision=2)
    children = dash_table.DataTable(columns=columns,data=data3.to_dict('records'))
    children2 = html.Div(children = str(len(data3))+' results found:')
    return children, children2


if __name__ == '__main__':
    app.run_server(debug=True)
